In [80]:
import json
import pandas as pd
import re
import ast

In [18]:
f = open('plumx_metrics_data.json')
data_json = json.load(f)

In [33]:
data_flat = pd.DataFrame()
for item in data_json:
    item_flat =  pd.json_normalize(item)
    data_flat = pd.concat([data_flat, item_flat], axis=0, join='outer')

In [35]:
data_flat.to_csv("plumx_metrics_cleaned.csv", index=False)

In [87]:
f = open('altmetric_data.json')
altmetrics_corpus = f.read()
altmetrics_corpus = altmetrics_corpus.replace('\\', '')
altmetrics_corpus = altmetrics_corpus.replace(',', ', ')
altmetrics_corpus = altmetrics_corpus.replace(':', ': ')
altmetrics_corpus = altmetrics_corpus[1:-1]

In [103]:
f = open('altmetrics_cleaner_data.json', "w")
f.write(altmetrics_corpus)
f.close()

In [104]:
f = open('altmetrics_cleaner_data.json')
data_json = json.load(f)

JSONDecodeError: Extra data: line 1 column 1917 (char 1916)

In [96]:
index = 0
item_start_index = None
item_end_index = None
total = len(altmetrics_corpus)
stack = 0
item_indices = list()
while(index < total):
    if(altmetrics_corpus[index]=='{' and stack == 0):
        item_start_index = index
        index+=1 
    elif(altmetrics_corpus[index]=='{' and stack != 0):
        stack+=1
        index+=1
    elif(altmetrics_corpus[index]=='}' and stack != 0):
        stack-=1
        index+=1
    elif(altmetrics_corpus[index]=='}' and stack == 0):
        item_end_index = index
        item_indices.append((item_start_index, item_end_index))
        index+=1
    else:
        index+=1   

In [98]:
len(item_indices)

88229

In [99]:
altmetrics_corpus[0]

'{'

In [100]:
item_indices[0]

(289, 388)